# Deploying a MedNIST Classifier with Ray

This notebook demos the process of deploying a network with Ray Serve as a web service. Ray provides various ways of deploying models with existing platforms like AWS or Azure but we'll focus on local deployment here since researchers are more likely to do this. Ray also provides other libraries for tuning, reinforcement learning, and distributed training in addition to deployment. This tutorial will use MedNIST classifier from the BentoML tutorial so please run at least the training component of that notebook first. The documentation on Ray Serve [start here](https://docs.ray.io/en/master/serve/index.html#rayserve), this notebook will be using the Pytorch specific functionality [discussed here](https://docs.ray.io/en/master/serve/tutorials/pytorch.html).

To start install the Ray Serve component:

In [ ]:
%pip install ray[serve]

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, tqdm]"

The imports for MONAI are the same as for the BentoML tutorial (assuming it's already installed):

In [7]:
import os
import io
from PIL import Image
import torch
import numpy as np
import requests

from ray import serve

from monai.apps import download_url
from monai.config import print_config
from monai.transforms import (
    AddChannel,
    Compose,
    ScaleIntensity,
    ToTensor,
)


print_config()

/usr/local/anaconda3/envs/py37/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


MONAI version: 0.5.1+2.gdde11000
Numpy version: 1.20.2
Pytorch version: 1.8.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: dde11000a7a10bcc9ccae76570cbb0e92ea23cf9

Optional dependencies:
Pytorch Ignite version: 0.4.4
Nibabel version: 3.2.1
scikit-image version: 0.18.1
Pillow version: 8.2.0
Tensorboard version: 2.5.0
gdown version: 3.12.2
TorchVision version: 0.9.1
ITK version: 5.1.2
tqdm version: 4.60.0
lmdb version: 1.2.1
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [ ]:
resource = "https://drive.google.com/uc?id=1zKRi5FrwEES_J-AUkM7iBJwc__jy6ct6"
dst = os.path.join("..", "bentoml", "classifier.zip")
if not os.path.exists(dst):
    download_url(resource, dst)

This class will represent the service for the model, which accepts an image sent as the body of a POST request and returns the class name in a JSON structure. Note that this class uses asyncio to define the `__call__` to be compatible with the server backend.

In [2]:
MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


class MedNISTClassifier:
    def __init__(self):
        # create the transform for normalizing the image data
        self.transform = Compose([AddChannel(), ScaleIntensity(), ToTensor()])
        # load the network on the CPU for simplicity and in eval mode
        self.net = torch.jit.load("../bentoml/classifier.zip", map_location="cpu").eval()

    async def __call__(self, request):
        image_bytes = await request.body()
        img = Image.open(io.BytesIO(image_bytes))
        img = np.array(img)
        image_tensor = self.transform(img)

        with torch.no_grad():
            outputs = self.net(image_tensor[None].float())

        _, output_classes = outputs.max(dim=1)

        return {"class_index": MEDNIST_CLASSES[output_classes[0]]}

Now the server is started and the classifier backend is associated with an endpoint, which is the route to the service relate to the server address.

In [3]:
client = serve.start()
client.create_backend("classifier", MedNISTClassifier)
client.create_endpoint("classifier", backend="classifier", route="/image_classify", methods=["POST"])

2021-03-26 14:36:24,683	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=25642) 2021-03-26 14:36:26,404	INFO http_state.py:67 -- Starting HTTP proxy with name 'oRDGQN:SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-node:10.246.179.34-0' on node 'node:10.246.179.34-0' listening on '127.0.0.1:8000'
(pid=25637) INFO:     Started server process [25637]
(pid=25642) 2021-03-26 14:36:34,799	INFO controller.py:178 -- Registering route '/image_classify' to endpoint 'classifier' with methods '['POST']'.


With the server running in another process we can send it a query with an image and get a response. By default the server will listen on port 8000.

In [4]:
image_bytes = open("./hand.jpg", "rb").read()

resp = requests.post("http://localhost:8000/image_classify", data=image_bytes)
print(resp.json())

(pid=25637) 2021-03-26 14:37:47,281	INFO router.py:248 -- Endpoint classifier doesn't exist, waiting for registration.


{'class_index': 'Hand'}


This can also be done on the command line with `curl`:

In [5]:
!curl -X POST "http://localhost:8000/image_classify" --data-binary "@hand.jpg"

{
  "class_index": "Hand"
}

Finally shut down the server:

In [6]:
client.shutdown()

2021-03-26 14:39:26,795	ERROR import_thread.py:89 -- ImportThread: Connection closed by server.
2021-03-26 14:39:26,800	ERROR worker.py:1109 -- listen_error_messages_raylet: Connection closed by server.
2021-03-26 14:39:26,803	ERROR worker.py:919 -- print_logs: Connection closed by server.


### Command Line Usage

Ray can be started on the command line. Since it operates as a cluster of nodes the first thing to do is create the head node locally then start the serve component:

In [7]:
%%bash

ray start --head
serve start

2021-03-26 14:54:40,757	INFO scripts.py:537 -- Local node IP: 10.246.179.34
2021-03-26 14:54:41,396	SUCC scripts.py:565 -- --------------------
2021-03-26 14:54:41,396	SUCC scripts.py:566 -- Ray runtime started.
2021-03-26 14:54:41,396	SUCC scripts.py:567 -- --------------------
2021-03-26 14:54:41,396	INFO scripts.py:569 -- Next steps
2021-03-26 14:54:41,396	INFO scripts.py:570 -- To connect to this Ray runtime from another node, run
2021-03-26 14:54:41,396	INFO scripts.py:574 --   ray start --address='10.246.179.34:6379' --redis-password='5241590000000000'
2021-03-26 14:54:41,396	INFO scripts.py:579 -- Alternatively, use the following Python code:
2021-03-26 14:54:41,396	INFO scripts.py:582 -- import ray
2021-03-26 14:54:41,396	INFO scripts.py:583 -- ray.init(address='auto', _redis_password='5241590000000000')
2021-03-26 14:54:41,396	INFO scripts.py:591 -- If connection fails, check your firewall settings and network configuration.
2021-03-26 14:54:41,396	INFO scripts.py:596 -- To te

2021-03-26 14:54:41,379	INFO services.py:1172 -- View the Ray dashboard at http://localhost:8265
2021-03-26 14:54:41,800	INFO worker.py:654 -- Connecting to existing Ray cluster at address: 10.246.179.34:6379
(pid=28386) 2021-03-26 14:54:42,165	INFO http_state.py:67 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-node:10.246.179.34-0' on node 'node:10.246.179.34-0' listening on '127.0.0.1:8000'
(pid=28423) INFO:     Started server process [28423]


A separate script with very similar code can then be used to add or replace the backend. This would be useful in an experimental setting where the server is running constantly in the background to which you can push updates quickly as you edit your script.

In [9]:
%%writefile mednist_classifier_start.py

import io
from PIL import Image
import torch
import numpy as np

import ray
from ray import serve

from monai.config import print_config
from monai.transforms import (
    AddChannel,
    Compose,
    ScaleIntensity,
    ToTensor,
)

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


class MedNISTClassifier:
    def __init__(self):
        self.transform = Compose([AddChannel(), ScaleIntensity(), ToTensor()])
        self.net = torch.jit.load("../bentoml/classifier.zip", map_location="cpu").eval()

    async def __call__(self, request):
        image_bytes = await request.body()
        img = Image.open(io.BytesIO(image_bytes))
        img = np.array(img)
        image_tensor = self.transform(img)

        with torch.no_grad():
            outputs = self.net(image_tensor[None].float())

        _, output_classes = outputs.max(dim=1)

        return {"class_index": MEDNIST_CLASSES[output_classes[0]]}


ray.init(address="auto")
client = serve.connect()

# remove previous instance of this backend if present
if "classifier" in client.list_backends():
    client.delete_endpoint("classifier")
    client.delete_backend("classifier")

client.create_backend("classifier", MedNISTClassifier)
client.create_endpoint("classifier", backend="classifier", route="/image_classify", methods=["POST"])

Writing mednist_classifier_start.py


The endpoint is then added by running the script:

In [10]:
!python mednist_classifier_start.py

2021-03-26 14:58:16,160	INFO worker.py:654 -- Connecting to existing Ray cluster at address: 10.246.179.34:6379
Exception ignored in: <function ActorHandle.__del__ at 0x7ff8c45b1ca0>
Traceback (most recent call last):
  File "/home/localek10/miniconda3/envs/monai/lib/python3.8/site-packages/ray/actor.py", line 769, in __del__
AttributeError: 'NoneType' object has no attribute 'global_worker'


And checked once again for response:

In [11]:
!curl -X POST "http://localhost:8000/image_classify" --data-binary "@hand.jpg"

{
  "class_index": "Hand"
}

Finally the service can be stopped:

In [12]:
!ray stop

Stopped all 19 Ray processes.
